<a href="https://colab.research.google.com/github/bs3537/DS-Unit-4-Sprint-1-NLP/blob/master/OK_Copy_TPOTClassifier_Spacy_NLP_whiskey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np


In [0]:
#USING VECTORIZATION BY SPACY NLP AND KERAS SEQUENTIAL NEURAL NETWORK FOR TRAINING ON TRAIN DATA AND MAKING PREDICTIONS
#DATASET: WHISKY RATINGS CLASSIFICATION FROM REVIEWS

In [0]:
import pandas as pd
train = pd.read_csv('https://raw.githubusercontent.com/bs3537/DS-Unit-4-Sprint-1-NLP/master/module3-document-classification/train.csv')

In [0]:
train['text'] = train['description'].str.strip('\n')

In [0]:
train2 = train.drop(columns=['description', 'id'])

In [0]:
train3 = train2.rename(columns={"ratingCategory": "label"})

In [0]:
train3.head(1)

,label,text
0,1,"Sometimes, when whisky is batched, a few lefto..."


In [0]:
from sklearn.model_selection import train_test_split

df_trn, df_val = train_test_split(train3, stratify = train3['label'], test_size = 0.20, random_state=42)

df_trn.shape, df_val.shape

((3269, 2), (818, 2))

In [0]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [0]:
doc = nlp("NLP is awesome!")

In [0]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [0]:
X = df_trn['text']
y_train = df_trn['label']
X_train_spacy = get_word_vectors(X)

In [0]:
#Val. data

X_val = df_val['text']
y_val = df_val['label']
X_val_spacy = get_word_vectors(X_val)

In [0]:
#TPOT AUTOML MODEL

In [0]:
from tpot import TPOTClassifier

In [0]:
tpot = TPOTClassifier(scoring='accuracy', random_state=42, n_jobs=-1, verbosity=1, generations=5, population_size=100, periodic_checkpoint_folder="intermediate_results")    #default populations and generations=100

In [0]:
from tqdm import tqdm_notebook as tqdm

In [0]:
%time
import timeit
times = []
scores = []
winning_pipes = []

# run 2 iterations
for x in range(1):
    start_time = timeit.default_timer()
    tpot.fit(np.array(X_train_spacy), y_train)
    elapsed = timeit.default_timer() - start_time
    times.append(elapsed)
    winning_pipes.append(tpot.fitted_pipeline_)
    scores.append(tpot.score(np.array(X_val_spacy), y_val))
    tpot.export('tpot_nlp_whisky.py')

# output results
times = [time/60 for time in times]
print('Times:', times)
print('Scores:', scores)   
print('Winning pipelines:', winning_pipes)

Wall time: 0 ns
Best pipeline: LinearSVC(input_matrix, C=5.0, dual=True, loss=hinge, penalty=l2, tol=0.001)
Times: [229.05956299999997]
Scores: [0.7591687041564792]
Winning pipelines: [Pipeline(memory=None,
         steps=[('linearsvc',
                 LinearSVC(C=5.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=42, tol=0.001,
                           verbose=0))],
         verbose=False)]


In [0]:
print(tpot.fitted_pipeline_)
#Winning pipeline is given below

Pipeline(memory=None,
         steps=[('linearsvc',
                 LinearSVC(C=5.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=42, tol=0.001,
                           verbose=0))],
         verbose=False)


In [0]:
tpot.export('whisky_tpot_digits_pipeline.py')

In [0]:
tpot.score(np.array(X_train_spacy), y_train)

0.7895380850412971

In [0]:
#predict on X_val
y_pred = tpot.predict(np.array(X_val_spacy))

In [0]:
from sklearn.metrics import accuracy_score
print("Validation accuracy = ", accuracy_score(y_pred, y_val))

Validation accuracy =  0.7591687041564792
